## SQL Querys


In [7]:
# Importing packages
import psycopg2
import pandas as pd
import numpy as np
from pandasql import sqldf # helpfull to query and use pandas df to results


In [4]:
## Base funcions

#Method to connect to DB
def connectDB():
    #create a connection
    conn = psycopg2.connect(
    host="localhost",
    database="ads",
    user="postgres",
    password="gremio")

    #create a cursor
    cur = conn.cursor()
    print("Connected")
    return conn, cur

#Selecting the data

def selectData(sql_select):
    #input = Query
    cur.execute(sql_select)
    data = cur.fetchall()

    #showing
    #x = input("Wanna print the data? Press 1 ==>> ____")
    #if x==1:
        #for line in data:
            #print(line)
    return data



In [ ]:

# Define a lambda function to translate your SQL queries to Python
pysqldf = lambda q: sqldf(q, globals())
# Play with your data using SQL. Change the value of the variable 'query' to yield different results
query = 'select * from world'
pysqldf(query)

In [12]:
conn, cur = connectDB()
data = selectData('select * from world')

Connected


## Exercises Week 3

<img src="capture.png">

## Question 1

Build the database in Figure 1. Consider the following constraints:
Values in the semester field (attribute) should be one of (’Q1’, ’Q2’, ’Q1’,
’Q2’, ’Q3’, ’Q4’)
Values of the year should be greater than 2000.

## Code to create tables

drop table Course Cascade;
drop table Students Cascade;
drop table Enrolled Cascade;

drop type semester;

create table Students ( sid integer , name text, major text, top_credits integer, gpa real,
					   primary key (sid) );


CREATE TYPE semester AS ENUM ('Q1', 'Q2', 'Q3', 'Q4');

create table Enrolled
( sid integer ,
 semester semester,
 cid integer, 
 year integer check(year > 2000),
 grade integer, 
 primary key (sid, cid, semester, year), 
 );


create table Course (cid integer, title text, dept_name text, credits integer, primary key (cid));

## Put Constraints
alter table Enrolled add CONSTRAINT fk_Students foreign key (sid) references Students(sid) on delete cascade;
alter table Enrolled add CONSTRAINT fk_Course foreign key (cid) references Course(cid) on delete cascade;
alter table Grades alter sid type integer using sid::integer;

## drop Constraints

alter table Enrolled drop constraint fk_students
alter table Enrolled drop constraint fk_Course

## Update Values

update Enrolled
set year = 2020
where year = 2001

update Students
set lastname = 'Franke789112345678901234567890' where students.name = 'Hans'

## change column type

alter table Grades alter sid type integer using sid::integer
alter table Students alter lastname type varchar(30)

## Creating Views

create view BestStudents (sid, name)
as select students.sid, students.name, students.gpa
from students
where students.gpa > (select AVG(students.gpa) from students)


## Question 2

Write queries to insert at least 4 records in each table. For the records tob
inserted in the Student and Course relations, insert them one-by-one but for
the records to be inserted in Enrolled relation, insert them in a single query.

## insert values
insert into Students (sid, name, major, top_credits, gpa) 	Values (1, 'Hans', 'ADS', 20, 4);
insert into Students (sid, name, major, top_credits, gpa) 	Values (2, 'Hakim', 'ADS', 20, 3);
insert into Students (sid, name, major, top_credits, gpa)	Values (3, 'Melkow', 'ADS', 20, 1);
insert into Students (sid, name, major, top_credits, gpa)	Values (4, 'Arno', 'ADS', 20, 4);

insert into Course (cid, title, dept_name, credits) values(1, 'ADS', 'Comp Science', 20);
insert into Course (cid, title, dept_name, credits) values(2, 'DataWrang', 'Comp Science', 30);
insert into Course (cid, title, dept_name, credits) values(3, 'Coloquium', 'Comp Science', 10);
insert into Course (cid, title, dept_name, credits) values(4, 'Tesis', 'Utrecht General', 10);

insert into Enrolled ( sid, semester, cid, Enrolled.year, grade)
	Values 1, 'Q2', 1, 2001, 20) , (2, 'Q1', 1, 2001, 20),
			(1, 'Q2', 3, 2001, 20) , (1, 'Q2', 3, 2001, 20)


## Question 3

The inclusion dependcy means that the values of a given attribute should
appear in another attribute. The referential integrity is a special case of inclusion dependency. Write a query that returns all the records which violates
the inclusion dependency between Enrolled.sid and Student.sid.


Query
if there is any return value, so there is a violation in insertion in Enrolled because of the foreign key constrain

select * from Enrolled where Enrolled.sid NOT IN (select students.sid from students)
select * from Enrolled where Enrolled.sid NOT IN (select Course.cid from Course)





## Question 4

Consider a new table for the grades of the studnets in the INFOMDWR Q1 2021.
This relation has the following schema (sid : string, grade_ME : real, grade_RE: real, grade_HO : real, grade_FI : real).
1. Write the SQL query that creates the table. Note that grade_ME cannot be null. If a student didn’t have a grade then it will be 0.
2. Instert a set of at least 4 records in the table. All the records shouldn’t contain values for the grade FI.
3. Write a query to return the records in which a student has grades in both grade RE and grade HO as these are violating the rule that a student can take only one additional exam with the main exam.
4. Write a query compute the grade FI as the average of either grade ME and grade RE or grade ME and grade HO.


#1 

create table Grades(
	sid integer,
	grade_ME real not null default 0,
	grade_RE real,
	grade_HO real,
	grade_FI real,
	primary key (sid, grade_ME)
);

#2

insert into Grades(sid, grade_me, grade_re, grade_ho)
	values(4, 9, null, 10), (2, 8, null, 9), (3, 4, 6, null)
    
this INSERT is just to prove constraint of  initial value as 0  

insert into Grades(sid)
	values(5)
    
#3

select * from Grades where (grade_re is not null) AND (grade_ho is not null)

#4 

Update Grades
set grade_fi = (grade_re + grade_me)/2
where grade_re is not null

Update Grades
set grade_fi = (grade_ho + grade_me)/2
where grade_ho is not null


## Question 5 

Consider the table in csv (’fdExample.csv’) file uploaded on the teams, if you
know that (First name → Gender), write a python script that can check all
the violation of such functional dependency.


In [98]:
import pandas as pd
df = pd.read_csv('fdExample.csv')

In [99]:
df.head()

First name Gender Department                              Department Name  \
0        Pam      F        POL                         Department of Police   
1      David      M        POL                         Department of Police   
2     Marsha      F        HHS      Department of Health and Human Services   
3    Godfred      M        COR                Correction and Rehabilitation   
4    Essayas      M        HCA  Department of Housing and Community Affairs   

                                            Division Assignment Category  \
0  MSB Information Management and Technology Divi...    Fulltime-Regular   
1         ISB Major Crimes Division Fugitive Section    Fulltime-Regular   
2      Adult Protective and Case Management Services    Fulltime-Regular   
3                         PRRS Facility and Security    Fulltime-Regular   
4                      Single Family Housing Program    Fulltime-Regular   

                Position Title Underfilled Job Title Date First Hired  
0  Office Services Coordinator                   NaN          9/22/86  
1        Master Police Officer                   NaN          9/12/88  
2             Social Worker IV                   NaN         11/19/89  
3       Resident Supervisor II                   NaN           5/5/14  
4      Planning Specialist III                   NaN           3/5/07

In [100]:
# Uncluster the Gender to Columns
df['M'] = df['Gender'] == "M"
df['F'] = df['Gender'] == "F"


In [115]:
# Split the DF and replace True and False to 1,0 to sum values later
df_firstname_gender = df[['First name' , 'M' , 'F']]
df_firstname_gender.replace({True:1, False:0}, inplace=True)
df_firstname_gender.head()

C:\Users\hansf\anaconda3\lib\site-packages\pandas\core\frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


First name  M  F
0        Pam  0  1
1      David  1  0
2     Marsha  0  1
3    Godfred  1  0
4    Essayas  1  0

In [118]:
#Groupby the same names and then sum the values
df2 = df_firstname_gender.groupby(by=df_firstname_gender['First name']).sum()
df2.head()

M  F
First name      
A'Hesh      1  0
A.          1  0
Aaron       9  0
Aarti       0  1
Abby        0  1

In [129]:
# Create a field that compares the values of name with M or F
import numpy as np
df2['check'] = np.where((df2['M'] > 0) & (df2['F'] > 0), "Error" , "OK")
df2[df2['check'] == 'Error'] 

M   F  check
First name                
Alan         14   1  Error
Alexis        1   1  Error
Andre        10   1  Error
Andrea        1  10  Error
Angel         1   2  Error
Antwan        1   1  Error
Carey         2   2  Error
Carroll       5   1  Error
Corey         9   1  Error
Dale          7   1  Error
Dana          2  10  Error
Danielle      1   8  Error
David       156   1  Error
Deon          1   1  Error
Dominique     2   1  Error
Donald       31   1  Error
Eric         43   1  Error
Erin          1  14  Error
Frederick     3   1  Error
Hilary        2   1  Error
Hui           1   1  Error
Jaime         1   1  Error
Jamie         4   5  Error
Jean          2   9  Error
Jerry         8   1  Error
Jesus         2   1  Error
Jin           1   1  Error
Juan         11   1  Error
Kelly         3   9  Error
Kerry         2   1  Error
Lee           8   1  Error
Leslie        2  13  Error
M             1   1  Error
Marion        1   1  Error
Maxie         1   1  Error
My            1   1  Error
Noel          1   1  Error
Randy         4   1  Error
Rene          3   1  Error
Ronnie        4   2  Error
Shannon       2   7  Error
Shawn         9   3  Error
Sidney        2   1  Error
Stacey        1   6  Error
Stacy         3   8  Error
Stefan        1   1  Error
Sun           1   1  Error
Taylor        3   1  Error
Terry         2   2  Error
Torrie        1   1  Error
Tracy         3   6  Error
Yu            1   1  Error

In [132]:
len(df2[df2['check'] == 'Error'] )

52

In [135]:
df2.head()

M  F check
First name            
A'Hesh      1  0    OK
A.          1  0    OK
Aaron       9  0    OK
Aarti       0  1    OK
Abby        0  1    OK

In [139]:
# Alternative solution, just count unique values
len( df[df['First name'] == 'Aarti'].Gender.unique() )

1

## Random code from Hakim



df.head()

In [3]:
import pandas as pd

In [29]:
## alternative solution 3
df = pd.read_csv('fdExample.csv')
grouped = df.groupby(df['First name']).nunique()
grouped[grouped['Gender'] == 2]


First name  Gender  Department  Department Name  Division  \
First name                                                              
Alan                 1       2           9                9        15   
Alexis               1       2           2                2         2   
Andre                1       2           5                5         9   
Andrea               1       2           9                9        11   
Angel                1       2           2                2         3   
Antwan               1       2           2                2         2   
Carey                1       2           4                4         4   
Carroll              1       2           6                6         6   
Corey                1       2           7                7        10   
Dale                 1       2           5                5         8   
Dana                 1       2           6                6        11   
Danielle             1       2           5                5         9   
David                1       2          20               20       103   
Deon                 1       2           2                2         2   
Dominique            1       2           2                2         2   
Donald               1       2          11               11        25   
Eric                 1       2          14               14        33   
Erin                 1       2           8                8        15   
Frederick            1       2           3                3         4   
Hilary               1       2           2                2         3   
Hui                  1       2           2                2         2   
Jaime                1       2           2                2         2   
Jamie                1       2           6                6         9   
Jean                 1       2           7                7         9   
Jerry                1       2           5                5         8   
Jesus                1       2           2                2         3   
Jin                  1       2           2                2         2   
Juan                 1       2           8                8        12   
Kelly                1       2           7                7        11   
Kerry                1       2           3                3         3   
Lee                  1       2           4                4         7   
Leslie               1       2          10               10        15   
M                    1       2           2                2         2   
Marion               1       2           2                2         2   
Maxie                1       2           2                2         2   
My                   1       2           2                2         2   
Noel                 1       2           2                2         2   
Randy                1       2           4                4         5   
Rene                 1       2           2                2         4   
Ronnie               1       2           5                5         6   
Shannon              1       2           5                5         9   
Shawn                1       2           6                6        11   
Sidney               1       2           3                3         3   
Stacey               1       2           3                3         6   
Stacy                1       2           7                7        10   
Stefan               1       2           2                2         2   
Sun                  1       2           2                2         2   
Taylor               1       2           4                4         4   
Terry                1       2           4                4         4   
Torrie               1       2           2                2         2   
Tracy                1       2           5                5         9   
Yu                   1       2           2                2         2   

            Assignment Category  Position Title  Underfilled Job Titl

In [146]:
#showing one example!
print(df[['First name','Gender']][df['First name'] == 'Alan'])

     First name Gender
476        Alan      M
1140       Alan      M
1344       Alan      M
3529       Alan      M
4062       Alan      M
5687       Alan      M
5924       Alan      M
6262       Alan      F
6676       Alan      M
6783       Alan      M
7042       Alan      M
7517       Alan      M
7750       Alan      M
7765       Alan      M
9036       Alan      M


In [27]:
len(grouped[grouped['Gender'] == 2])

52

## Proving that a foreing key dont have to be a primary key


In [ ]:
create table test_foreign (
	id integer primary key,
	name text
);

create table test_primary (
	cid integer primary key,
	id integer
);

alter table test_primary add constraint primary_foreing foreign key (id) references test_foreign(id);


insert into test_foreign values (1, 'hi') , (2,'bye');

insert into test_primary values (1, 1) , (2, 1);

select * from test_primary;

select * from test_foreign;
